In [3]:
import os

In [4]:
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Red-Wine-Quality-Prediction\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Red-Wine-Quality-Prediction'

## Entity Configuration

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

## Configuration Manager

In [8]:
from wine.constants import *
from wine.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name   
        )

        return model_trainer_config

## Component Creation

In [10]:
import pandas as pd
import os
from wine import logger
from sklearn.linear_model import ElasticNet
import joblib

[2024-04-14 13:10:26,466: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-04-14 13:10:26,467: INFO: utils: NumExpr defaulting to 8 threads.]


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

## Execution/Pipeline

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-04-14 13:10:27,291: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-14 13:10:27,293: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-14 13:10:27,295: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-14 13:10:27,296: INFO: common: created directory at: artifacts]
[2024-04-14 13:10:27,297: INFO: common: created directory at: artifacts/model_trainer]
